# Data Management
Zuallererst wurden die Daten von https://gol.gg/players/list/season-S10/split-ALL/tournament-ALL/ als Excel-Tabelle herunter geladen. Aus dieser Tabelle wurden Bilder entfernt und die Spalten-Bezeichnungen hinzugefügt. 
Die Abkürzungen bedeuten:<br>
- KDA = Kill Death Assist <br>
- CSM = Average Creep Score per Minute per Game <br>
- GPM = Average Gold per Minute per Games <br>
- KP = Average Kill Participation per Game<br>
- DMG = Average share of team's total damage to champions per game<br>
- DPM = Average damage to champions per minute per game<br>
- VSPM = Average vision score per minute per game<br>
- WPM = Wards per minute per Game<br>
- WCPM = Wards cleared per minute per game<br>
- VWPM = vision wards per minute per game<br>
- GD@15 = Average Gold Difference at 15 minutes<br>
- CSD@15 = Average Creep Score Difference at 15 minutes<br>
- XPD@15 = Average Experience Points Difference at 15 minutes<br>
- FB = Firstblood Participation<br>
- FB Victim = First blood Victim<br>
- Penta Kills = Total Pentakills<br>
- Solo Kills = Total Kills without Teammate Participation<br>

In [1]:
import pandas as pd
import os

In [2]:
pd.set_option("display.max_columns", None)

---

Laden der Rohdaten in Pandas (Hier nur zu Doku zwecken. In Zukunft die bereits bearbeiteten Daten nutzen!)<br>
In rawPath kommt der Pfad zur Excel Tabelle.

In [3]:
rawPath = "./RawData.xlsx"
rawdf = pd.read_excel(rawPath, engine='openpyxl')
index = rawdf[rawdf['Player'].isna()].index
rawdf.drop(index=index, inplace=True)
rawdf.dropna(axis=1, inplace=True)
rawdf.head()

,Player,Position,Games,Win Rate,KDA,Avg Kills,Avg Deaths,Avg Assists,CSM,GPM,KP%,DMG%,DPM,VSPM,Avg WPM,Avg WCPM,Avg VWPM,GD@15,CSD@15,XPD@15,FB%,FB Victim,Penta Kills,Solo Kills
0,369,TOP,129.0,66.7%,3.6,2.8,2.4,5.9,8.2,400,57.2%,22.4,469,1.05,0.39,0.21,0.14,137,4,5,8.5%,10.1%,0,24
1,3z,TOP,26.0,42.3%,1.8,1.9,3.4,4.1,7.8,365,57.8%,26.2,424,1.02,0.49,0.17,0.2,-483,-6,-245,7.7%,30.8%,0,5
2,705,TOP,21.0,47.6%,2.3,1.6,3.1,5.6,7.2,341,61.8%,21.2,359,0.69,0.46,0.15,0.15,-631,0,311,4.8%,0%,0,-
3,ACD,TOP,45.0,48.9%,2.5,1.7,3.0,5.7,7.3,359,56.6%,22.9,435,1.1,0.52,0.19,0.22,136,4,62,24.4%,11.1%,0,15
4,ADD,TOP,17.0,35.3%,2.2,2.2,3.8,6.3,6.6,341,67.5%,20.7,452,1.39,0.49,0.21,0.19,-356,-10,-317,17.6%,0%,0,-


Die Spalte "Position" wird entfernt, da sie immer gleich ist. <br> Spalte "Player" wird entfernt da sie nicht benötigt wird. <br> Die Spalte "Penta Kills" wird entfernt, weil sie quasi keine Daten beinhaltet und dementsprechend für die Analyse irrelevant ist. <br> Die Spalte "VSPM" wird entfernt, da diese Daten erst seit Season 8 existieren und somit für die Hälfte des Datensatzes irrelevant ist.

In [4]:
df = rawdf.drop(["Position","Player","Penta Kills", "VSPM"], axis = 1)
df.head()

,Games,Win Rate,KDA,Avg Kills,Avg Deaths,Avg Assists,CSM,GPM,KP%,DMG%,DPM,Avg WPM,Avg WCPM,Avg VWPM,GD@15,CSD@15,XPD@15,FB%,FB Victim,Solo Kills
0,129.0,66.7%,3.6,2.8,2.4,5.9,8.2,400,57.2%,22.4,469,0.39,0.21,0.14,137,4,5,8.5%,10.1%,24
1,26.0,42.3%,1.8,1.9,3.4,4.1,7.8,365,57.8%,26.2,424,0.49,0.17,0.2,-483,-6,-245,7.7%,30.8%,5
2,21.0,47.6%,2.3,1.6,3.1,5.6,7.2,341,61.8%,21.2,359,0.46,0.15,0.15,-631,0,311,4.8%,0%,-
3,45.0,48.9%,2.5,1.7,3.0,5.7,7.3,359,56.6%,22.9,435,0.52,0.19,0.22,136,4,62,24.4%,11.1%,15
4,17.0,35.3%,2.2,2.2,3.8,6.3,6.6,341,67.5%,20.7,452,0.49,0.21,0.19,-356,-10,-317,17.6%,0%,-


Alle "-" in Spalte "Solo Kills" werden zu Nullen konvertiert

In [5]:
df["Solo Kills"] = df["Solo Kills"].replace("-", "0")

Zeilen aussortieren, bei denen Datenpunkte fehlen. 

In [6]:
for col in list(df):
        df.drop(index = df[df[col] == "-"].index,inplace = True)

Alle Prozentzahlen werden zu floats konvertiert

In [7]:
percentageColumns = ["Win Rate", "KP%", "DMG%", "FB%", "FB Victim"]
for col in percentageColumns:
    df[col] = df[col].str.rstrip('%').astype('float') / 100.0

Alle Spieler die weniger als 10 Spiele gespielt haben werden entfernt, da diese aufgrund der geringen Spielanzahl keine statistisch relevanten Daten haben können. <br>
Alle Spieler mit einer Winrate von 0 % werden entfernt, da diese als Anomalien das Neuronale Netzwerk verwirren und damit beeinträchtigen könnten.

In [8]:
indexFewGames = df[df["Games"] < 10].index
df.drop(indexFewGames, inplace = True)
indexLowWinRate = df[ df['Win Rate'] == 0].index
df.drop(indexLowWinRate , inplace=True)

Die Daten werden als ".csv" zur Wiederverwendung gespeichert

In [9]:
df.to_csv("cleanData.csv", index = False)

---

## Label
### Win Rate
Die Spalte Win Rate gibt an wie viele Prozent aller gespielten Spiele der Spieler gewonnen hat. Das Neuronale Netzwerk soll aus allen anderen Metriken versuchen diese Win Rate zu berechnen.

## Features
### Games
Die Metrik Games gibt an wie viele Spiele des jeweiligen Spielers in der Datenbank aufgezeichnet wurden. Je mehr Spiele ein   Spieler gespielt hat, desto mehr sagen alle anderen Statistiken aus. Außerdem tendiert die Win Rate bei einer großen Spielanzahl gegen 50 %.
### KDA
Die Metrik KDA gibt das Verhältnis zwischen Kills (Champion-Tötungen), Deaths (Toden des eigenen Champion) und Assists (Unterstützung bei Kills von gegnerischen Champions) wieder. Sie wird durch die Formel (K + A)/MAX(1,D) berechnet. In Worten: Die Kills addiert mit den Assists wird durch die Größere Zahl aus 1 und den Deaths geteilt. Dies ist eine der wichtigsten Metriken, da sie die wichtigsten Aspekte des Spiels zusammenfast.
### Avg Kills
Die Spalte Avg Kills gibt die Durchschnittlichen Kills eines Spielers pro Spiel wieder. Sie ergänzt die KDA, da man durch die KDA nicht genau einsehen kann, wie hoch die einzelnen Metriken die zur Berechnung der KDA benötigt werden, sind. Die Spalten Avg Deaths und Avg Assists geben genau das gleiche wieder, nur für die Durchschnittlichen Deaths und Assists.
### CSM
Die Metrik CSM spiegelt wieder, wieviele Creeps ein Spieler im Durchschnitt pro Minute getötet hat. Dies ist wichtig weil Creeps-Töten die primäre Goldquelle im Spiel sind und so eine hohe CSM für viel Gold steht.
### GPM
GPM gibt das durchschnittliche Goldeinkommen eines Spielers pro wieder. Gold wird durch das töten von Creeps und Champions sowie das Einreißen von Türmen.
### KP%
KP% gibt an, an wie viel Prozent aller Kills des Teams Spieler durchschnittlich beteiligt ist. Darein zählen sowohl die eigenen Kills als auch die Assists.
### DMG%
Die Metrik DMG% gibt an wieviel Prozent des Schadens an gegnerischen Champions des Teams der Spieler durchschnittlich beteiligt war.
### DPM
Die Metrik DPM spiegelt wider, wieviel Schaden ein Spieler besonders starken gegnerischen Einheiten, sogenannten "Champions" durchschnittlich über seine Spielzeit hinweg zufügt. Durch einen diesbezüglich hohen Wert kann vermutet werden, dass ein Spieler einen großen Beitrag zu der Zerstörung der gegnerischen Champions beiträgt und somit auch viele Tode der eigenen Mitspieler verhindert, was generell eine gute Eigenschaft eines Spielers ist.
### Avg WPM
Die Spalte "Avg WPM" gibt einen Durchschnitt der Metrik "Wards Per Minute" an. Im speziellen spiegelt dieser Wert also wider, wie schnell und häufig ein Spieler "Wards" aller Art (eine stationäre Einheit, die bestimmte Gegner im Umfeld angreift) plaziert. Während eine effektive Plazierung der Wards natürlich auch wichtig ist, ist ein hoher Durschnittswert natürlich positiver als eine niedrige Platzierungsrate.
### Avg WCPM
Die Spalte "Avg WCPM" gibt einen Durchschnittswert bezüglich der Rate an, mit welcher ein Spieler generische Wards (eine stationäre Einheit, die bestimmte Gegner im Umfeld angreift) zerstört an. Im speziellen spiegelt dieser Wert also wider, wie schnell und häufig ein Spieler Wards aller Art aus dem Weg räumt und somit Schaden an seinem Team verhindert.
### Avg VWPM
Die Spalte "Avg VWPM" gibt einen Durchschnitt der Metrik "Vision Wards Per Minute" an. Im speziellen spiegelt dieser Wert also wider, wie schnell und häufig ein Spieler im Speziellen "Vision Wards" (eine stationäre Einheit, die bestimmte Gegner im Umfeld angreift) plaziert. Während eine effektive Plazierung der Vision Wards natürlich auch wichtig ist, ist ein hoher Durschnittswert natürlich positiver als eine niedrige Platzierungsrate.
### GD@15
GD@15 gibt an, wieviel mehr oder weniger Gold ein Spieler im Vergleich zu dem gleichpositionierten Gegenspieler nach 15 Minuten hat. Eine hohe Anzahl an Gold zeigt auf, dass ein Spieler viele Kills o.ä. getätigt hat und somit einen guten Beitrag zum Teamergebnis leistet. Ist die Difference/Unterschied zum Enemy Laner also hoch bzw. positiv, so hat der Spieler im Durschnitt all seiner Runden eher überdurschnittlich viel Gold gesammelt.
### CSD@15
CSD@15 gibt die durchschnittliche Creep Score Difference nach 15 Minuten an. Der Creep Score gibt in League of Legends generell die Anzahl der Creeps bzw. einfacher zu tötenden Bots an, die ein Champion getötet hat. Ein hoher Creep Score pro Minute ist sehr positiv, weshalb die Creep Score Difference, bzw. der Abstand zum CS anderer Spieler ebenfalls positiv und nicht negativ sein sollte.
### XPD@15
Der Wert XPD@15 beschreibt, wieviel weniger oder mehr Erfahrungspunkte ein Spieler durch erfolgreiche Kills etc. nach 15 Minuten gesammelt hat. Die XP-Difference nach 15 Minuten misst ist ein Wert der einen Überblick über erfolgreiche Handlungen und die Gesamtperformance des Spielers gibt. Der XP Abstand zu Anderen Spielern, also XPD, sollte bei einer überdurchschnittlichen Performance positiv sein.
### FB
Die Spalte FB besagt, welchen Teil der Spieler zu dem ersten Kill einer Runde beigetragen hat. Diese "First Bloos Participation" kann für positiv sowie negativ sein. Zwar trägt der Spieler dazu bei, dass das Team vermehrt Gold sammelt, jedoch geht er auch die Risiken einer aggressiven Spielweise ein.
### FB 
Die Spalte "FB Victim" beschreibt, inwieviel Prozent der Fälle der jeweilige Spieler in einer Runde als erstes gestorben ist. Dieser Prozentsatz sollte möglichst niedrig sein.
### Solo Kills
Die Metrik Solo Kills beschreibt, wieviele Kills insgesamt getätigt wurden, ohne dass Teammitglieder hierzu beigegetragen haben. Dieser Wert und das Verhältnis zu der Gesamtanzahl der Kills haben eine hohe Aussagekraft darüber, wie eigenständig und Teamunabhängig ein Spieler ist.